# CC-NA Annotation Sample Generation

This notebook was used to generate the sample of code-contributing non-authors used in our brief extended analysis in the QSS paper.

In [ ]:
import time
from pathlib import Path

import pandas as pd
from gh_tokens_loader import GitHubTokensCycler
from ghapi.all import GhApi
from sqlalchemy import create_engine, text
from tqdm import tqdm

from rs_graph.db import models as db_models
from rs_graph.db.constants import PROD_DATABASE_FILEPATH

EXTRA_DATA_DIR = Path.cwd().parent / "extra-data"

# Get db engine for production database
db_conn = create_engine(f"sqlite:///{PROD_DATABASE_FILEPATH}")

In [2]:
def read_table(table: str) -> pd.DataFrame:
    return pd.read_sql(text(f"SELECT * FROM {table}"), db_conn)


# Read all data from database
doc_repo_links = read_table(db_models.DocumentRepositoryLink.__tablename__)
researchers = read_table(db_models.Researcher.__tablename__)
devs = read_table(db_models.DeveloperAccount.__tablename__)
documents = read_table(db_models.Document.__tablename__)
document_contributors = read_table(db_models.DocumentContributor.__tablename__)
repositories = read_table(db_models.Repository.__tablename__)
repository_contributors = read_table(db_models.RepositoryContributor.__tablename__)
topics = read_table(db_models.Topic.__tablename__)
document_topics = read_table(db_models.DocumentTopic.__tablename__)
dataset_sources = read_table(db_models.DatasetSource.__tablename__)
researcher_dev_links = read_table(
    db_models.ResearcherDeveloperAccountLink.__tablename__
)
document_alternate_dois = read_table(db_models.DocumentAlternateDOI.__tablename__)

# Drop all "updated_datetime" and "created_datetime" columns
for df in [
    doc_repo_links,
    researchers,
    devs,
    documents,
    document_contributors,
    repositories,
    repository_contributors,
    topics,
    document_topics,
    dataset_sources,
    researcher_dev_links,
]:
    df.drop(columns=["updated_datetime", "created_datetime"], inplace=True)

# Specifically drop doc_repo_links "id" column
# It isn't used and will get in the way later when we do a lot of joins
doc_repo_links.drop(columns=["id"], inplace=True)

# Construct reduced doc_repo_links
original_doc_repo_links_len = len(doc_repo_links)
doc_repo_links = doc_repo_links.drop_duplicates(subset=["document_id"], keep=False)
doc_repo_links = doc_repo_links.drop_duplicates(subset=["repository_id"], keep=False)

# Reduce other tables to only documents / repositories in the updated doc_repo_links
documents = documents[documents["id"].isin(doc_repo_links["document_id"])]
repositories = repositories[repositories["id"].isin(doc_repo_links["repository_id"])]
document_contributors = document_contributors[
    document_contributors["document_id"].isin(documents["id"])
]
repository_contributors = repository_contributors[
    repository_contributors["repository_id"].isin(repositories["id"])
]
document_topics = document_topics[document_topics["document_id"].isin(documents["id"])]

# Reduce researchers and devs to only those in the
# updated document_contributors and repository_contributors
researchers = researchers[
    researchers["id"].isin(document_contributors["researcher_id"])
]
devs = devs[devs["id"].isin(repository_contributors["developer_account_id"])]
researcher_dev_links = researcher_dev_links[
    (
        researcher_dev_links["researcher_id"].isin(researchers["id"])
        & researcher_dev_links["developer_account_id"].isin(devs["id"])
    )
]

# Sort document topics and keep first
document_topics = document_topics.sort_values("score", ascending=False)
document_topics = document_topics.drop_duplicates(subset=["document_id"], keep="first")

# Create document, document topic merged table
merged_document_topics = pd.merge(
    document_topics, topics, left_on="topic_id", right_on="id"
)

# Create basic merged tables
merged_document_contributor_doc_repo_links = pd.merge(
    document_contributors, doc_repo_links, left_on="document_id", right_on="document_id"
)
merged_repository_contributor_doc_repo_links = pd.merge(
    repository_contributors,
    doc_repo_links,
    left_on="repository_id",
    right_on="repository_id",
)

# Compute stats for data sources
data_source_stats = []
for _, data_source in dataset_sources.iterrows():
    # Get total article-repo pairs
    data_source_stats.append(
        {
            "data_source": data_source["name"],
            "n_article_repo_pairs": len(
                doc_repo_links[doc_repo_links["dataset_source_id"] == data_source["id"]]
            ),
            "n_authors": merged_document_contributor_doc_repo_links.loc[
                merged_document_contributor_doc_repo_links["dataset_source_id"]
                == data_source["id"]
            ]["researcher_id"].nunique(),
            "n_devs": merged_repository_contributor_doc_repo_links.loc[
                merged_repository_contributor_doc_repo_links["dataset_source_id"]
                == data_source["id"]
            ]["developer_account_id"].nunique(),
        }
    )

# Create topic merged tables
merged_doc_repo_links_topics = pd.merge(
    doc_repo_links, document_topics, left_on="document_id", right_on="document_id"
).merge(topics, left_on="topic_id", right_on="id")
merged_doc_repo_links_topics_document_contributors = pd.merge(
    merged_doc_repo_links_topics,
    document_contributors,
    left_on="document_id",
    right_on="document_id",
)
merged_doc_repo_links_topics_repository_contributors = pd.merge(
    merged_doc_repo_links_topics,
    repository_contributors,
    left_on="repository_id",
    right_on="repository_id",
)

# Compute stats for domains
domain_stats = []
for domain in merged_doc_repo_links_topics.domain_name.unique():
    # Get total article-repo pairs
    domain_stats.append(
        {
            "domain": domain,
            "n_article_repo_pairs": len(
                merged_doc_repo_links_topics[
                    merged_doc_repo_links_topics["domain_name"] == domain
                ]
            ),
            "n_authors": merged_doc_repo_links_topics_document_contributors.loc[
                merged_doc_repo_links_topics_document_contributors["domain_name"]
                == domain
            ]["researcher_id"].nunique(),
            "n_devs": merged_doc_repo_links_topics_repository_contributors.loc[
                merged_doc_repo_links_topics_repository_contributors["domain_name"]
                == domain
            ]["developer_account_id"].nunique(),
        }
    )

# Create document merged tables
merged_doc_repo_links_documents = pd.merge(
    doc_repo_links, documents, left_on="document_id", right_on="id"
)
merged_doc_repo_links_documents_document_contributors = pd.merge(
    merged_doc_repo_links_documents,
    document_contributors,
    left_on="document_id",
    right_on="document_id",
)
merged_doc_repo_links_documents_repository_contributors = pd.merge(
    merged_doc_repo_links_documents,
    repository_contributors,
    left_on="repository_id",
    right_on="repository_id",
)

# Compute stats for document types
# This isn't a standard data pull
# In short:
# - pairs from PLOS are "research articles"
# - pairs from JOSS are "software articles"
# - pairs from SoftwareX are "software articles"
# - pairs from Papers with Code / ArXiv are "pre-prints"
#   UNLESS they have been published in a journal
# All of those should be easy to assert / apply a label to with the exception
# of Papers with Code / ArXiv pre-prints that have been published in a journal
# In that case, we need to look at the existing document type in the database
# If the document type is "preprint" use preprint, otherwise, if it's anything else,
# use "research article"

# Create a "reduced_doc_types" dataframe with document_id and "reduced_doc_type"
# columns
reduced_doc_types_rows = []
# We can use the "reduced_doc_types" dataframe to calculate the stats

# Iter over data sources even though we are looking for doc types
for _, data_source in dataset_sources.iterrows():
    # Get total article-repo pairs
    doc_type = None
    if data_source["name"] in ["plos", "joss", "softwarex"]:
        if data_source["name"] == "plos":
            doc_type = "research article"
        else:
            doc_type = "software article"

        # Add all document_ids to reduced_doc_types_rows
        reduced_doc_types_rows.extend(
            [
                {"document_id": doc_id, "reduced_doc_type": doc_type}
                for doc_id in doc_repo_links[
                    (doc_repo_links["dataset_source_id"] == data_source["id"])
                ]["document_id"]
            ]
        )

    # Handle PwC
    else:
        # Get preprint pairs
        preprint_pairs = merged_doc_repo_links_documents[
            (merged_doc_repo_links_documents["dataset_source_id"] == data_source["id"])
            & (merged_doc_repo_links_documents["document_type"] == "preprint")
        ]

        # Add all document_ids to reduced_doc_types_rows
        reduced_doc_types_rows.extend(
            [
                {"document_id": doc_id, "reduced_doc_type": "preprint"}
                for doc_id in preprint_pairs["document_id"]
            ]
        )

        # Get research article pairs
        # This is the same just inverted to != "preprint"
        research_article_pairs = merged_doc_repo_links_documents[
            (merged_doc_repo_links_documents["dataset_source_id"] == data_source["id"])
            & (merged_doc_repo_links_documents["document_type"] != "preprint")
        ]

        # Add all document_ids to reduced_doc_types_rows
        reduced_doc_types_rows.extend(
            [
                {"document_id": doc_id, "reduced_doc_type": "research article"}
                for doc_id in research_article_pairs["document_id"]
            ]
        )

# Create reduced_doc_types dataframe
reduced_doc_types = pd.DataFrame(reduced_doc_types_rows)

In [ ]:
# Create subset documents
docs_w_1_citation = documents.loc[documents["cited_by_count"] >= 1].copy()

# Take sample?
# docs_w_1_citation = docs_w_1_citation.sample(frac=0.1, random_state=12)

# Subset to only certain columns
docs_w_1_citation = docs_w_1_citation[
    [
        "id",
        "publication_date",
        "cited_by_count",
        "fwci",
        "is_open_access",
    ]
]

# Rename id to document_id
docs_w_1_citation = docs_w_1_citation.rename(columns={"id": "document_id"})

# Merge repository id in
docs_w_1_citation = docs_w_1_citation.merge(
    doc_repo_links[
        [
            "document_id",
            "repository_id",
        ]
    ],
    left_on="document_id",
    right_on="document_id",
)

# Merge in document details (domain, document type)
docs_w_1_citation = (
    docs_w_1_citation.merge(
        document_topics[["document_id", "topic_id"]],
        left_on="document_id",
        right_on="document_id",
    )
    .merge(
        repositories[["id", "creation_datetime", "last_pushed_datetime"]],
        left_on="repository_id",
        right_on="id",
    )
    .drop(
        columns=["id"],
    )
    .merge(
        topics[["id", "domain_name"]],
        left_on="topic_id",
        right_on="id",
    )
    .drop(
        columns=["id", "topic_id"],
    )
    .merge(
        reduced_doc_types,
        left_on="document_id",
        right_on="document_id",
    )
    .rename(
        columns={
            "domain_name": "domain",
            "reduced_doc_type": "article_type",
        }
    )
)

# Drop any documents that have more than one repository (and vice versa)
docs_w_1_citation = docs_w_1_citation.drop_duplicates(
    subset=["document_id"], keep=False
)
docs_w_1_citation = docs_w_1_citation.drop_duplicates(
    subset=["repository_id"], keep=False
)

# Iter over articles and get the team composition info
team_composition_rows = []
cc_na_individuals = []
for _, row in tqdm(
    docs_w_1_citation.iterrows(),
    total=len(docs_w_1_citation),
):
    # Get the number of authors
    author_ids = document_contributors.loc[
        document_contributors["document_id"] == row["document_id"]
    ]["researcher_id"].unique()
    n_authors = len(author_ids)

    # Get the number of devs
    dev_ids = repository_contributors.loc[
        repository_contributors["repository_id"] == row["repository_id"]
    ]["developer_account_id"].unique()
    n_devs = len(dev_ids)

    # Get the set of dev_researcher_links for the devs
    dev_researcher_links = researcher_dev_links.loc[
        researcher_dev_links["developer_account_id"].isin(dev_ids)
    ].sort_values("predictive_model_confidence", ascending=False)

    # Drop duplicates by dev_id (keeping first)
    # as we may have accidently matched the same author to the multiple devs
    dev_researcher_links = dev_researcher_links.drop_duplicates(
        subset=["researcher_id"],
        keep="first",
    )

    # Drop any dev researcher links that have less than 97% confidence
    dev_researcher_links = dev_researcher_links.loc[
        dev_researcher_links["predictive_model_confidence"] >= 0.97
    ]

    # Get the number of devs who aren't authors
    n_non_author_devs = 0
    for dev_id in dev_ids:
        dev_researcher_ids = dev_researcher_links.loc[
            dev_researcher_links["developer_account_id"] == dev_id
        ]["researcher_id"].unique()

        if len(dev_researcher_ids) > 3:
            continue

        if not any(
            dev_researcher_id in author_ids for dev_researcher_id in dev_researcher_ids
        ):
            cc_na_individuals.append(
                {
                    "document_id": row["document_id"],
                    "repository_id": row["repository_id"],
                    "n_authors": n_authors,
                    "n_devs": n_devs,
                    "developer_account_id": dev_id,
                    "researcher_ids_count": len(dev_researcher_ids),
                    "researcher_ids": dev_researcher_ids.tolist(),
                }
            )

# Create cc_na_individuals dataframe
cc_na_individuals = pd.DataFrame(cc_na_individuals)

# Merge with cc_na_individuals
cc_na_individuals = cc_na_individuals.merge(
    docs_w_1_citation,
    left_on=["document_id", "repository_id"],
    right_on=["document_id", "repository_id"],
)

# Filter out papers with less than 3 authors or 1 dev
cc_na_individuals = cc_na_individuals.loc[cc_na_individuals["n_authors"] >= 3]
cc_na_individuals = cc_na_individuals.loc[cc_na_individuals["n_devs"] >= 1]

# Convert datetimes to datetime
cc_na_individuals["publication_date"] = pd.to_datetime(
    cc_na_individuals["publication_date"],
    utc=True,
)
cc_na_individuals["last_pushed_datetime"] = pd.to_datetime(
    cc_na_individuals["last_pushed_datetime"],
    utc=True,
)
cc_na_individuals["creation_datetime"] = pd.to_datetime(
    cc_na_individuals["creation_datetime"],
    utc=True,
)

# Create a "days_since_last_push" column
cc_na_individuals["days_from_publication_to_last_push"] = (
    cc_na_individuals["last_pushed_datetime"] - cc_na_individuals["publication_date"]
).dt.days

# Must have a push within 90 days of publication
cc_na_individuals_no_push_after_pub = cc_na_individuals.loc[
    cc_na_individuals["days_from_publication_to_last_push"] <= 90
].copy()

# Get the number of authors which would be the 97th percentile
n_authors_97th_percentile = cc_na_individuals_no_push_after_pub["n_authors"].quantile(
    0.97
)

# Remove rows that are greater than 97th percentile for total authors
cc_na_individuals_no_push_after_pub = cc_na_individuals_no_push_after_pub.loc[
    cc_na_individuals_no_push_after_pub["n_authors"]
    <= cc_na_individuals_no_push_after_pub["n_authors"].quantile(0.97)
]

cc_na_individuals_no_push_after_pub[
    [
        "document_id",
        "repository_id",
        "developer_account_id",
        "researcher_ids_count",
        "researcher_ids",
    ]
]

100%|██████████| 88551/88551 [02:00<00:00, 735.26it/s]


,document_id,repository_id,developer_account_id,researcher_ids_count,researcher_ids
8,21,21,39,0,[]
17,55,55,89,0,[]
47,82,81,164,0,[]
74,126,125,261,0,[]
81,138,137,283,0,[]
...,...,...,...,...,...
79430,157265,153544,130890,0,[]
79456,157277,153555,130890,0,[]
79457,157279,153557,130890,0,[]
79458,157280,153558,130890,0,[]


In [4]:
cc_na_individuals_no_push_after_pub.loc[
    cc_na_individuals_no_push_after_pub["researcher_ids_count"] > 1
][
    [
        "document_id",
        "repository_id",
        "developer_account_id",
        "researcher_ids_count",
        "researcher_ids",
    ]
].sort_values(
    by="researcher_ids_count", ascending=False
)

,document_id,repository_id,developer_account_id,researcher_ids_count,researcher_ids
26443,43020,41089,40097,3,"[72424, 44008, 281382]"
18545,25305,24280,1576,3,"[209976, 304662, 91404]"
39664,71206,68300,28978,3,"[83498, 174985, 252680]"
23085,35663,34110,10027,3,"[97043, 104851, 293211]"
57340,110223,106641,4140,3,"[81282, 1093, 87934]"
...,...,...,...,...,...
36462,64096,61366,34238,2,"[60280, 12572]"
36614,64322,61591,7040,2,"[185747, 183477]"
38099,68257,65398,80595,2,"[6365, 16309]"
38514,68939,66064,30927,2,"[53189, 195198]"


In [5]:
len(
    cc_na_individuals_no_push_after_pub.loc[
        cc_na_individuals_no_push_after_pub["researcher_ids_count"] > 1
    ]
) / len(cc_na_individuals_no_push_after_pub)

0.030234166584329612

In [6]:
len(
    cc_na_individuals_no_push_after_pub.loc[
        cc_na_individuals_no_push_after_pub["researcher_ids_count"] > 1
    ]
)

306

In [7]:
cc_na_individuals_no_push_after_pub.loc[
    cc_na_individuals_no_push_after_pub["researcher_ids_count"] == 0
][
    [
        "document_id",
        "repository_id",
        "developer_account_id",
        "researcher_ids_count",
        "researcher_ids",
    ]
].sample(
    10
)

,document_id,repository_id,developer_account_id,researcher_ids_count,researcher_ids
41438,75381,72373,72535,0,[]
54988,105139,101615,129245,0,[]
28915,47837,45803,75708,0,[]
31627,53557,51256,81968,0,[]
12154,10266,9957,25628,0,[]
35957,63047,60353,91978,0,[]
18353,24868,23848,46682,0,[]
54631,104535,101030,128711,0,[]
21587,32648,31200,56851,0,[]
26907,44112,42153,71137,0,[]


In [ ]:
tokens_file = Path.cwd().parent / ".github-tokens.yml"
gh_tokens_cycler = GitHubTokensCycler(tokens_file)

# Take a random sample of 20 rows
# Then, get the:
# - document id
# - document DOI
# - repository id
# - repository URL
# - developer account id
# - developer account URL


def prep_for_annotation(row: pd.Series) -> dict:
    # Get document details
    doc_details = documents.loc[documents["id"] == row["document_id"]].iloc[0]
    doc_doi = doc_details["doi"]
    doc_doi_url = f"https://doi.org/{doc_doi}"

    # Get repository details
    repo_details = repositories.loc[repositories["id"] == row["repository_id"]].iloc[0]
    repo_owner = repo_details["owner"]
    repo_name = repo_details["name"]
    repo_url = f"https://github.com/{repo_owner}/{repo_name}"

    # Get developer account details
    dev_details = devs.loc[devs["id"] == row["developer_account_id"]].iloc[0]
    dev_username = dev_details["username"]
    dev_url = f"https://github.com/{dev_username}"

    # Dev commit URL
    dev_commit_url = (
        f"https://github.com/{repo_owner}/{repo_name}/commits?author={dev_username}"
    )

    # Get devs contributions to repository
    results = None
    try:
        call_iter = 0
        while call_iter < 9:
            api = GhApi(
                token=next(gh_tokens_cycler),
            )
            results = api.repos.get_contributors_stats(
                owner=repo_owner,
                repo=repo_name,
            )

            # If the results are empty, wait and try again
            if len(results) == 0:
                time.sleep(20)
                call_iter += 1
                results = None
                continue

            # If we have results, break the loop
            break

        # If we have results, filter for the developer account
        if results is None:
            print(f"No results found for {repo_url} after {call_iter + 1} iterations.")
            raise ValueError(
                f"No results found for {repo_url} after {call_iter + 1} iterations."
            )

        # Print number of call iterations
        print(f"Number of API call iterations required: {call_iter + 1}")

        # Get the total number of commits, additions, and deletions for the repo
        # Iter over each contributor, and iter over each week in "weeks", sum all values
        total_repo_commits = 0
        total_repo_additions = 0
        total_repo_deletions = 0
        for contributor in results:
            for week in contributor["weeks"]:
                total_repo_commits += week["c"]
                total_repo_additions += week["a"]
                total_repo_deletions += week["d"]

        # If we have results, filter for the developer account
        filtered_to_contributor = [
            contributor
            for contributor in results
            if contributor["author"]["login"] == dev_username
        ]
        if len(filtered_to_contributor) > 0:
            contributor = filtered_to_contributor[0]
            total_commits = sum(week["c"] for week in contributor["weeks"])
            total_additions = sum(week["a"] for week in contributor["weeks"])
            total_deletions = sum(week["d"] for week in contributor["weeks"])
        else:
            total_commits = 0
            total_additions = 0
            total_deletions = 0

        # Add all of these details to the results
        return {
            "document_id": row["document_id"],
            "document_doi": doc_doi_url,
            "repository_id": row["repository_id"],
            "repository_url": repo_url,
            "developer_account_id": row["developer_account_id"],
            "developer_account_url": dev_url,
            "developer_contribution_url": dev_commit_url,
            "n_commits": total_commits,
            "n_additions": total_additions,
            "n_deletions": total_deletions,
            "total_repo_commits": total_repo_commits,
            "total_repo_additions": total_repo_additions,
            "total_repo_deletions": total_repo_deletions,
        }

    except Exception as e:
        print(f"Error processing {repo_url} for {dev_username}: {e}")
        return {
            "document_id": row["document_id"],
            "document_doi": doc_doi_url,
            "repository_id": row["repository_id"],
            "repository_url": repo_url,
            "developer_account_id": row["developer_account_id"],
            "developer_account_url": dev_url,
            "developer_contribution_url": dev_commit_url,
        }


# Take a random sample of 20 rows
sampled_rows = cc_na_individuals_no_push_after_pub.drop_duplicates(
    "developer_account_id", keep="first"
).sample(n=20, random_state=12)

annotation_ready = []
for _, row in tqdm(
    sampled_rows.iterrows(),
    total=len(sampled_rows),
):
    # Prepare the row for annotation
    annotation_ready.append(prep_for_annotation(row))

    # Convert to DataFrame
    annotated_df = pd.DataFrame(annotation_ready)
    annotated_df.to_csv(EXTRA_DATA_DIR / "cc-na-individuals-sample.csv", index=False)

# Convert to DataFrame
annotated_df = pd.DataFrame(annotation_ready)
annotated_df.to_csv(EXTRA_DATA_DIR / "cc-na-individuals-sample.csv", index=False)

Expired GitHub Tokens: 1


  0%|          | 0/20 [00:00<?, ?it/s]

Expired GitHub Tokens: 1


  5%|▌         | 1/20 [00:00<00:09,  2.07it/s]

Number of API call iterations required: 1


...

Number of API call iterations required: 2


 95%|█████████▌| 19/20 [16:50<00:54, 54.66s/it]

Number of API call iterations required: 5


100%|██████████| 20/20 [17:52<00:00, 53.63s/it]

Number of API call iterations required: 4


In [ ]:
# Take a random sample of 200 rows
sampled_rows = cc_na_individuals_no_push_after_pub.drop_duplicates(
    "developer_account_id", keep="first"
).sample(n=200, random_state=12)

annotation_ready = []
for _, row in tqdm(
    sampled_rows.iterrows(),
    total=len(sampled_rows),
):
    # Prepare the row for annotation
    annotation_ready.append(prep_for_annotation(row))

    # Convert to DataFrame
    annotated_df = pd.DataFrame(annotation_ready)
    annotated_df.to_csv(EXTRA_DATA_DIR / "cc-na-individuals-full.csv", index=False)

# Convert to DataFrame
annotated_df = pd.DataFrame(annotation_ready)
annotated_df.to_csv(EXTRA_DATA_DIR / "cc-na-individuals-full.csv", index=False)

  0%|          | 1/200 [00:00<00:47,  4.16it/s]

Number of API call iterations required: 1


  1%|          | 2/200 [00:00<01:03,  3.13it/s]

Number of API call iterations required: 1


  2%|▏         | 3/200 [00:00<01:07,  2.90it/s]

...

Number of API call iterations required: 1


100%|█████████▉| 199/200 [2:42:39<00:39, 39.04s/it]

Number of API call iterations required: 5


100%|██████████| 200/200 [2:44:22<00:00, 49.31s/it]

Number of API call iterations required: 6
